In [ ]:
from skimage import segmentation, io
import os
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import pandas as pd
model = keras.models.load_model('./model/model.h5')

In [ ]:
#path_to_images = './data/eval_xai_method/dog_resized'
path_to_images ='./data/debug'
all_images = os.listdir(path_to_images)

In [ ]:
num_images = 1

In [ ]:
images_chosen = np.random.choice(all_images, size=num_images)

In [ ]:
prediction_list = []
image_name = images_chosen[0]
image_path = os.path.join(path_to_images, image_name)
img = io.imread(image_path)
img_norm = img/255
img_batch = np.expand_dims(img_norm, axis=0)
pred = model.predict(img_batch)
prediction_list.append(pred)
io.imshow(img)
print(prediction_list)

In [ ]:
print(img.shape)

In [ ]:
superpixels = segmentation.quickshift(img_norm, kernel_size=4, max_dist=10, ratio=.3)
num_superpixels = np.unique(superpixels).shape[0]
print(num_superpixels)
io.imshow(segmentation.mark_boundaries(img_norm, superpixels, color=(1,0,0), mode='inner'))

In [ ]:
class ImagesManager:
    
    def __init__(self, location, images_extension):
        self.location = location
        self.images_extension = images_extension
    
    def get_images_names(self):
        all_data = os.listdir(self.location)
        images_names = [x for x in all_data if x.split('.')[-1] == self.images_extension]
        return images_names
    
    def __len__(self):
        return len(self.get_images_names())

In [ ]:
class MaskManager:
    
    def __init__(self, location, image_name):
        self.location = location
        self.image_name = image_name
        
    def get_mask(self, kernel_size, max_dist, ratio):
        
        full_path = os.path.join(self.location, self.image_name)
        img = io.imread(full_path)
        mask = segmentation.quickshift(img, kernel_size=kernel_size, max_dist=max_dist, ratio=ratio)
            
        return mask

In [ ]:
class FeaturesManager:
    
    def __init__(self, location, image_name, mask, normalise_features, num_channels):
        self.location = location
        self.image_name = image_name
        self.mask = mask
        self.features_properties = [
            {'name': 'total_pixels', 'multiple_values': False, 'fun': self.get_total_pixels},
            {'name': 'num_superpixel', 'multiple_values': False, 'fun': self.get_num_superpixel},
            {'name': 'image_name', 'multiple_values': False, 'fun': self.get_image_name},
            {'name': 'mean_channel', 'multiple_values': True, 'fun': self.get_mean},
            {'name': 'std_channel', 'multiple_values': True, 'fun': self.get_std},
            {'name': 'channels_correlation', 'multiple_values': True, 'fun': self.get_channels_corraltion},
        ]
        self.normalise_features = normalise_features
        self.num_channels = num_channels
        
    def get_num_superpixel(self, num_superpixel, *arg, **kwargs):
        return num_superpixel
    
    def get_total_pixels(self, superpixel, *args, **kwargs):
        return superpixel.shape[0]
    
    def get_image_name(self, *args, **kwargs):
        return self.image_name
    
    def get_mean(self, superpixel, *args, **kwargs):
        if self.normalise_features:
            return np.mean(superpixel/255, axis=0)
        else:
            return np.mean(superpixel, axis=0)
    
    def get_std(self, superpixel, *args, **kwargs):
        if self.normalise_features:
            return np.std(superpixel/255, axis=0)
        else:
            return np.std(superpixel, axis=0)
        
    def get_channels_corraltion(self, superpixel, *args, **kwargs):
        
        superpixel_norm = superpixel
        superpixel_shape = superpixel_norm.shape
        
        if self.normalise_features:
            superpixel_norm = superpixel/255
        
        if superpixel_shape[0] > 1 and superpixel_shape[1]:
            corr_matrix = np.corrcoef(superpixel_norm, rowvar=False)
            idx = np.triu_indices(n=self.num_channels, m=self.num_channels, k=1)
            corr_vector = corr_matrix[idx]
        
        else:
            n = self.num_channels
            len_vector = int(n*(n-1)/2)
            corr_vector = np.repeat(None, len_vector)
            
        return corr_vector

            
    def create_data_frame(self, dict_features):
        
        df = pd.DataFrame(dict_features)
            
        for feature in self.features_properties:
            if feature['multiple_values']:
                feature_name = feature['name']
                values = df[feature_name].to_list()
                len_values = len(values[0])
                names = [feature_name + str(i) for i in range(len_values)]
                df[names] = values
                df.drop(columns=feature_name, inplace=True)

        return df
    
    def get_features(self):
        
        full_path = os.path.join(self.location, self.image_name)
        img = io.imread(full_path)
        
        mask = self.mask
        
        # Obtain features
        unique_superpixels = np.unique(self.mask)
    
        dict_features = {}
        
        for index in unique_superpixels:
            superpixel = img[mask == index]
            for feature in self.features_properties:
                feature_name = feature['name']
                feature_values = feature['fun'](superpixel=superpixel, num_superpixel=index, img=img, mask=mask)
                
                if feature_name in dict_features.keys():
                    dict_features[feature_name].append(feature_values)
                else:
                    dict_features[feature_name] = [feature_values]
            
        pd_features = self.create_data_frame(dict_features)
        return pd_features

In [ ]:
def get_superpixels_information(path_to_images, images_extension='jpg',
                                normalise_features=False, num_channels=3,
                                kernel_size=4, max_dist=10, ratio=.3):
    
    images_manager = ImagesManager(location=path_to_images, images_extension=images_extension)
    images_names = images_manager.get_images_names()
    
    df_features_list = []
    dict_masks = {}
    
    for image_name in images_names:
        masks_manager = MaskManager(location=path_to_images, image_name=image_name)
        mask = masks_manager.get_mask(kernel_size=kernel_size, max_dist=max_dist, ratio=ratio)
        dict_masks[image_name] = mask
        features_manager = FeaturesManager(location=path_to_images, image_name=image_name, 
                                           mask=mask, normalise_features=True, num_channels=num_channels)
        df_features = features_manager.get_features()
        df_features_list.append(df_features)
    
    return pd.concat(df_features_list), dict_masks

In [ ]:
#featurizantion, masks = get_superpixels_information('./data/debug')
df_features, masks = get_superpixels_information('./data/debug', normalise_features=True)

In [ ]:
print(df_features.head(2))

In [ ]:
import numpy as np
xarr = np.random.normal(size=(6, 3))
mat = np.corrcoef(xarr, rowvar=False)
print(mat)

In [ ]:
n_col=3
idx = np.arange(start=0, stop=9, step=n_col+1)
corr_flat = mat.flatten()
print(np.delete(corr_flat, idx))